In [45]:
import pandas as pd
import numpy as np
import folium
import geopandas
import branca
import warnings

import psutil
import tracemalloc

In [46]:
# Start tracking memory allocations
tracemalloc.start()

In [47]:
# Importing file path
data_filepath = '/home/ubuntu/data_analysis/gdhi.xlsx'

In [48]:
# Reading the GDHI datasheet
gdhi = pd.read_excel(data_filepath, 
                     sheet_name='Table 1',
                     header=1)

gdhi.head()

,ITL,ITL code,Region name,1997,1998,1999,2000,2001,2002,2003,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,UK,UK,United Kingdom,617647,632254,662795,699578,729614,749989,776614,...,1074751,1124684,1167426,1239740,1263676,1300608,1358278,1409660,1402455,1453047
1,Other,TLB,England,527846,541181,568150,599552,625121,641457,663758,...,919814,964172,1002232,1068525,1090397,1122348,1173887,1218276,1210984,1255871
2,ITL1,TLC,North East,23372,23421,24195,25373,26505,26974,27823,...,36945,37950,39014,40530,41146,42139,43447,45032,45397,46749
3,ITL2,TLC1,Tees Valley and Durham,10411,10342,10789,11310,11843,12004,12323,...,16378,16778,17345,17957,18200,18661,19252,19904,20137,20743
4,ITL3,TLC11,Hartlepool and Stockton-on-Tees,2442,2429,2569,2688,2827,2919,3012,...,4093,4150,4295,4413,4483,4589,4714,4892,4947,5101


In [49]:
# Reading gdhi per head sheet
gdhi_perhead = pd.read_excel(data_filepath, 
                     sheet_name='Table 3',
                     header=1)

gdhi_perhead.head()

,ITL,ITL code,Region name,1997,1998,1999,2000,2001,2002,2003,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,UK,UK,United Kingdom,10592,10812,11294,11880,12343,12633,13022,...,16871,17544,18073,19041,19249,19694,20445,21104,20907,21679
1,Other,TLB,England,10847,11085,11587,12178,12642,12912,13295,...,17195,17900,18452,19504,19729,20179,20971,21644,21414,22213
2,ITL1,TLC,North East,9101,9146,9487,9976,10435,10617,10952,...,14197,14537,14898,15442,15606,15933,16346,16866,16935,17663
3,ITL2,TLC1,Tees Valley and Durham,9046,8995,9403,9860,10335,10479,10750,...,13901,14200,14642,15120,15259,15601,16026,16504,16637,17293
4,ITL3,TLC11,Hartlepool and Stockton-on-Tees,9114,9027,9498,9870,10318,10617,10919,...,14372,14505,14963,15343,15523,15850,16229,16809,16986,17615


In [51]:
"""
To find basic info about our dataframe, 

# for basic information about, 
# column and row size, 
# non-null row count, 
# column types, and 
# column names
df.info() 

# for basic statistics about the numeric columns
df.describe()

# for checking the duplicated rows, 
duplicate = df.duplicated(keep='first')

# to count the number of duplicate rows,
duplicate.value_counts() # No TRUE value in the result represents no duplicated rows

# to check for duplicated column names
pd.Series(df.columns.duplicated()).value_counts() # No TRUE value in the result represents no duplicated rows

# Check unique_valued_columns,
# - like a primary key column in a relational database
df.columns[df.nunique(axis=0) == df.shape[0]]

"""

"\nTo find basic info about our dataframe, \n\n# for basic information about, \n# column and row size, \n# non-null row count, \n# column types, and \n# column names\ndf.info() \n\n# for basic statistics about the numeric columns\ndf.describe()\n\n# for checking the duplicated rows, \nduplicate = df.duplicated(keep='first')\n\n# to count the number of duplicate rows,\nduplicate.value_counts() # No TRUE value in the result represents no duplicated rows\n\n# to check for duplicated column names\npd.Series(df.columns.duplicated()).value_counts() # No TRUE value in the result represents no duplicated rows\n\n# Check unique_valued_columns,\n# - like a primary key column in a relational database\ndf.columns[df.nunique(axis=0) == df.shape[0]]\n\n"

In [52]:
gdhi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Data columns (total 28 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ITL          234 non-null    object
 1   ITL code     234 non-null    object
 2   Region name  234 non-null    object
 3   1997         234 non-null    int64 
 4   1998         234 non-null    int64 
 5   1999         234 non-null    int64 
 6   2000         234 non-null    int64 
 7   2001         234 non-null    int64 
 8   2002         234 non-null    int64 
 9   2003         234 non-null    int64 
 10  2004         234 non-null    int64 
 11  2005         234 non-null    int64 
 12  2006         234 non-null    int64 
 13  2007         234 non-null    int64 
 14  2008         234 non-null    int64 
 15  2009         234 non-null    int64 
 16  2010         234 non-null    int64 
 17  2011         234 non-null    int64 
 18  2012         234 non-null    int64 
 19  2013         234 non-null    

In [55]:
# Memory check for gdhi 
# Using pd.memory_usage(deep=True)

gdhi_memory_usage = gdhi_perhead.memory_usage(deep=True).sum() / 1024
print(f"gdhi_memory_usage using pd.memory_usage(deep=True): \n{gdhi_memory_usage:.2f} KBs")

gdhi_memory_usage using pd.memory_usage(deep=True): 
85.55 KBs


In [56]:
# Grouping the dataframe according to 'ITL'

def ITL_groups(df):

    # check the input type
    if not isinstance(df, pd.DataFrame):
        raise ValueError (f'{type(df)}is not a valid input type. Input must be pandas.DataFrame')

    groups = df.groupby(by='ITL')

    # Creating a dict of dfs 
    ITL_group = {name:data for name, data in groups}

    # printing the keys so we can understand what possible keys are in the dict created
    print(ITL_group.keys())
    
    return ITL_group

In [57]:
# Call ITL_groups function
gdhi_ITLgroups = ITL_groups(gdhi)
gdhi_perhead_ITLgroups = ITL_groups(gdhi_perhead)


dict_keys(['ITL1', 'ITL2', 'ITL3', 'Other', 'UK'])
dict_keys(['ITL1', 'ITL2', 'ITL3', 'Other', 'UK'])


In [58]:
# GDHI region name and 2021 values in a new df
def spinoff_df(df, 
               ITLregion: str='ITL1', 
               columns: str='ITL code',
              ):

    if ITLregion not in ('ITL1', 'ITL2', 'ITL3'):
        raise ValueError (f'{ITlregion} not supported. Valid inputs are ITL1, ITL2 or ITL3.')

    if not all(col in df[ITLregion].columns for col in columns):
        raise ValueError (f'Some columns in {columns} not in the df. Please give valid columns list')

    spinoff_df_ = df[ITLregion][columns]
    spinoff_df_.reset_index(drop=True, inplace=True)

    return spinoff_df_


In [60]:
# Call the spinoff_df function
GDHI_ITL1 = spinoff_df(df=gdhi_ITLgroups, ITLregion='ITL1', columns=['ITL code', '2021'])
GDHI_perhead_ITL1 = spinoff_df(df=gdhi_perhead_ITLgroups, ITLregion='ITL1', columns=['ITL code', '2021'])

# Merging both GDHI and GDHI_perhead
merged_GDHI_ITL1 = GDHI_ITL1.merge(GDHI_perhead_ITL1,
                                   how='left',
                                  left_on='ITL code',
                                  right_on='ITL code')

merged_GDHI_ITL1.rename(columns = {'2021_x':'GDHI_2021',
                                  '2021_y': 'GDHI_perhead_2021'
                                  }, inplace=True
                       )

merged_GDHI_ITL1.head()

,ITL code,GDHI_2021,GDHI_perhead_2021
0,TLC,46749,17663
1,TLD,140015,18864
2,TLE,100657,18363
3,TLF,92505,18956
4,TLG,110549,18566


In [61]:
# ITL shape files directory
ITL1_zipfile = 'zip:///home/ubuntu/data_analysis/ukshapefiles/International_Territorial_Level_1_January_2021_UK_BFE_2022_-9025318669378992679.zip'
ITL2_zipfile = 'zip:///home/ubuntu/data_analysis/ukshapefiles/International_Territorial_Level_2_January_2021_UK_BFE_V2_2022_-4735199360818908762.zip'
ITL3_zipfile = 'zip:///home/ubuntu/data_analysis/ukshapefiles/International_Territorial_Level_3_January_2021_UK_BFE_V3_2022_3795075883532745256.zip'


In [62]:
# use geopandas to read GeoJson file
def geopandasreadfile(shapefilepath):
    
    gdf = geopandas.read_file(shapefilepath)
    return gdf

In [63]:
# Call geopandasreadfile function to read ITL1 shape file
ITL1_gdf = geopandasreadfile(ITL1_zipfile)

ITL1_gdf.head()

,ITL121CD,ITL121NM,BNG_E,BNG_N,LONG,LAT,GlobalID,geometry
0,TLC,North East (England),417314,600356,-1.72888,55.2970,918a9395-2fdc-4fc3-95de-7f222e77333c,"MULTIPOLYGON (((424965.097 638594.304, 424963...."
1,TLD,North West (England),350014,506279,-2.77239,54.4494,43dd5b6c-4f3f-48b0-8274-0f8b7e51cbe9,"POLYGON ((357067.497 586765.6, 357029.1 586748..."
2,TLE,Yorkshire and The Humber,446902,448736,-1.28714,53.9326,b81e2cc4-ed0c-4ef8-b48e-0ac4e0f33ef1,"POLYGON ((478799 519215.1, 478803.5 519208.9, ..."
3,TLF,East Midlands (England),477659,322635,-0.84969,52.7957,dc88930f-9fd4-4216-8c9a-4fa9051b5deb,"POLYGON ((512572.598 414469.499, 512619.702 41..."
4,TLG,West Midlands (England),386294,295477,-2.20358,52.5570,fb1bcf9e-2430-4568-a68d-6eb3dfbb3965,"POLYGON ((402840.498 368845.598, 402862.397 36..."


In [64]:
# Merging gdf with GDHI df
gdf_GDHI_ITL1_merged = ITL1_gdf.merge(merged_GDHI_ITL1, how='left', 
                                left_on='ITL121CD', right_on='ITL code')

gdf_GDHI_ITL1_merged.head()

,ITL121CD,ITL121NM,BNG_E,BNG_N,LONG,LAT,GlobalID,geometry,ITL code,GDHI_2021,GDHI_perhead_2021
0,TLC,North East (England),417314,600356,-1.72888,55.2970,918a9395-2fdc-4fc3-95de-7f222e77333c,"MULTIPOLYGON (((424965.097 638594.304, 424963....",TLC,46749,17663
1,TLD,North West (England),350014,506279,-2.77239,54.4494,43dd5b6c-4f3f-48b0-8274-0f8b7e51cbe9,"POLYGON ((357067.497 586765.6, 357029.1 586748...",TLD,140015,18864
2,TLE,Yorkshire and The Humber,446902,448736,-1.28714,53.9326,b81e2cc4-ed0c-4ef8-b48e-0ac4e0f33ef1,"POLYGON ((478799 519215.1, 478803.5 519208.9, ...",TLE,100657,18363
3,TLF,East Midlands (England),477659,322635,-0.84969,52.7957,dc88930f-9fd4-4216-8c9a-4fa9051b5deb,"POLYGON ((512572.598 414469.499, 512619.702 41...",TLF,92505,18956
4,TLG,West Midlands (England),386294,295477,-2.20358,52.5570,fb1bcf9e-2430-4568-a68d-6eb3dfbb3965,"POLYGON ((402840.498 368845.598, 402862.397 36...",TLG,110549,18566


In [65]:
# Using the Branca linear colormap function, 
# Created colormap ranging between min and max values in the 2021 column.

def branca_colormap(df, 
                    column: str,
                    colors_: list()=["#e0f3db", "#7bccc4", "#4eb3d3", "#2b8cbe", "#084081"],
                    caption_: str='Values'):
    
    colormap_ = branca.colormap.LinearColormap(
        vmin=df[column].min(),
        vmax=df[column].max(),
        colors=colors_,
        caption=caption_,
    )

    return colormap_


In [66]:
# Call branca_colormap function
GDHI_ITL1_colormap = branca_colormap(gdf_GDHI_ITL1_merged, column='GDHI_2021', caption_='GDHI values in millions')

GDHI_ITL1_colormap

In [67]:
# Folium map for GDHI and GDHI_perhead

def folium_map(df, 
               key_value: dict,
               colormap_colors: list()=["#e0f3db", "#7bccc4", "#4eb3d3", "#2b8cbe", "#084081"],
               colormap_caption: str=None,
               filename: str='UK_GDHI_file.html'):

    if df.empty:
        raise ValueError (f'Cannot perform operations on an empty dataframe')

    key, value = None, None
    
    key = list(key_value.keys())[0]
    value = list(key_value.values())[0]

    if not key and not value:
        raise ValueError (f'Key value pair must not be empty')

    # Call branca_colormap function to retrieve colormap
    colormap_ = branca_colormap(df=df, column=value, colors_= colormap_colors, caption_=colormap_caption)

    # Create a map object
    m = folium.Map(location=[55,-5],
                  zoom_start=5,
                   tiles = 'Cartodb Positron'
                  )

    # Define tooltip for GeoJson
    tooltip = folium.GeoJsonTooltip(
        fields=[key, value], 
        aliases=["ITL Region:", "Value(in Millions):"], 
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    )
    
    # Insert style function
    def style_function_(feature):
        return {
            "fillColor": colormap_(feature["properties"][value]) 
            if feature["properties"][value] is not None 
            else "transparent",
            "color": "black",
            "fillOpacity": 1.0,
            "weight": 0.3,
        }

    # Insert highlight function
    def highlight_function_(feature):
        return {
            "fillColor": colormap_(feature["properties"][value]) 
            if feature["properties"][value] is not None
            else "transparent",
            "color": "black",    
            "weight": 3,       
            "fillOpacity": 1.0,
        }
    
    # Insert GeoJson layer
    geo = folium.GeoJson(
        df,
        style_function = style_function_,
        highlight_function = highlight_function_,
        tooltip=tooltip,
    ).add_to(m)
    
    colormap_.add_to(m)
    
    m.save(filename)
    print(f'Map saved in the directory.')
    return None


In [68]:
# Call folium_map function for GDHI_ITL1
GDHI_ITL1_map = folium_map(df= gdf_GDHI_ITL1_merged, 
                          colormap_caption='GDHI_values in millions',
                          key_value={'ITL121NM':'GDHI_2021'},
                          filename='UK_GDHI_2021_ITL1.html')

GDHI_ITL1_map

Map saved in the directory.


In [69]:
# Call folium_map function for ITL1
GDHI_perhead_ITL1_map = folium_map(df= gdf_GDHI_ITL1_merged, 
                          colormap_caption='GDHI_values in millions',
                          key_value={'ITL121NM':'GDHI_perhead_2021'},
                          filename='UK_GDHI_perhead_2021_ITL1.html')

GDHI_perhead_ITL1_map

Map saved in the directory.


In [70]:
# ITL2 map

In [71]:
# Call the spinoff_df function
GDHI_ITL2 = spinoff_df(df=gdhi_ITLgroups, ITLregion='ITL2', columns=['ITL code', '2021'])
GDHI_perhead_ITL2 = spinoff_df(df=gdhi_perhead_ITLgroups, ITLregion='ITL2', columns=['ITL code', '2021'])

# Merging both GDHI and GDHI_perhead
merged_GDHI_ITL2 = GDHI_ITL2.merge(GDHI_perhead_ITL2,
                                   how='left',
                                  left_on='ITL code',
                                  right_on='ITL code')

merged_GDHI_ITL2.rename(columns = {'2021_x':'GDHI_2021',
                                  '2021_y': 'GDHI_perhead_2021'
                                  }, inplace=True
                       )

merged_GDHI_ITL2.head()

,ITL code,GDHI_2021,GDHI_perhead_2021
0,TLC1,20743,17293
1,TLC2,26005,17969
2,TLD1,10056,20079
3,TLD3,52202,18199
4,TLD4,27791,18142


In [72]:
# Call geopandasreadfile function to read ITL2 shape file
ITL2_gdf = geopandasreadfile(ITL2_zipfile)

ITL2_gdf.head()

,ITL221CD,ITL221NM,BNG_E,BNG_N,LONG,LAT,geometry
0,TLC1,Tees Valley and Durham,414643,532242,-1.77440,54.6850,"POLYGON ((416954.896 558226.798, 416959.398 55..."
1,TLC2,"Northumberland, and Tyne and Wear",398898,602530,-2.01892,55.3168,"MULTIPOLYGON (((424965.097 638594.304, 424963...."
2,TLD1,Cumbria,341868,527389,-2.90218,54.6383,"POLYGON ((357067.497 586765.599, 357029.1 5867..."
3,TLD3,Greater Manchester,377940,401102,-2.33407,53.5061,"POLYGON ((391363.101 420849.902, 391421.401 42..."
4,TLD4,Lancashire,363669,441026,-2.55392,53.8642,"POLYGON ((370171.702 482434.604, 370185.002 48..."


In [73]:
# Merging gdf with GDHI df
gdf_GDHI_ITL2_merged = ITL2_gdf.merge(merged_GDHI_ITL2, how='left', 
                                left_on='ITL221CD', right_on='ITL code')

gdf_GDHI_ITL2_merged.head()

,ITL221CD,ITL221NM,BNG_E,BNG_N,LONG,LAT,geometry,ITL code,GDHI_2021,GDHI_perhead_2021
0,TLC1,Tees Valley and Durham,414643,532242,-1.77440,54.6850,"POLYGON ((416954.896 558226.798, 416959.398 55...",TLC1,20743,17293
1,TLC2,"Northumberland, and Tyne and Wear",398898,602530,-2.01892,55.3168,"MULTIPOLYGON (((424965.097 638594.304, 424963....",TLC2,26005,17969
2,TLD1,Cumbria,341868,527389,-2.90218,54.6383,"POLYGON ((357067.497 586765.599, 357029.1 5867...",TLD1,10056,20079
3,TLD3,Greater Manchester,377940,401102,-2.33407,53.5061,"POLYGON ((391363.101 420849.902, 391421.401 42...",TLD3,52202,18199
4,TLD4,Lancashire,363669,441026,-2.55392,53.8642,"POLYGON ((370171.702 482434.604, 370185.002 48...",TLD4,27791,18142


In [74]:
# Call folium_map function for GDHI_ITL2
GDHI_ITL2_map = folium_map(df= gdf_GDHI_ITL2_merged, 
                          colormap_caption='GDHI_values in millions',
                          key_value={'ITL221NM':'GDHI_2021'},
                          filename='UK_GDHI_2021_ITL2.html')

GDHI_ITL2_map

Map saved in the directory.


In [75]:
# Call folium_map function for GDHI_perhead_ITL2
GDHI_perhead_ITL2_map = folium_map(df= gdf_GDHI_ITL2_merged, 
                          colormap_caption='GDHI_values in millions',
                          key_value={'ITL221NM':'GDHI_perhead_2021'},
                          filename='UK_GDHI_perhead_2021_ITL2.html')

GDHI_perhead_ITL2_map

Map saved in the directory.


In [76]:
# ITL3 map

In [77]:
# Call the spinoff_df function
GDHI_ITL3 = spinoff_df(df=gdhi_ITLgroups, ITLregion='ITL3', columns=['ITL code', '2021'])
GDHI_perhead_ITL3 = spinoff_df(df=gdhi_perhead_ITLgroups, ITLregion='ITL3', columns=['ITL code', '2021'])

# Merging both GDHI and GDHI_perhead
merged_GDHI_ITL3 = GDHI_ITL3.merge(GDHI_perhead_ITL3,
                                   how='left',
                                  left_on='ITL code',
                                  right_on='ITL code')

merged_GDHI_ITL3.rename(columns = {'2021_x':'GDHI_2021',
                                  '2021_y': 'GDHI_perhead_2021'
                                  }, inplace=True
                       )

merged_GDHI_ITL3.head()

,ITL code,GDHI_2021,GDHI_perhead_2021
0,TLC11,5101,17615
1,TLC12,4672,16664
2,TLC13,1989,18376
3,TLC14,8982,17228
4,TLC21,6553,20380


In [78]:
# Call geopandasreadfile function to read ITL2 shape file
ITL3_gdf = geopandasreadfile(ITL3_zipfile)

ITL3_gdf.head()

,ITL321CD,ITL321NM,BNG_E,BNG_N,LONG,LAT,GlobalID,geometry
0,TLC11,Hartlepool and Stockton-on-Tees,444952,522016,-1.30587,54.5914,94a02515-1576-4743-8e3d-a843ae72fc4a,"POLYGON ((447213.9 537036.104, 447228.798 5370..."
1,TLC12,South Teesside,461718,519597,-1.04695,54.5678,f9353bdb-f621-4560-9f58-58c0e888a0c9,"POLYGON ((455834.103 528110.599, 455845.003 52..."
2,TLC13,Darlington,428029,515649,-1.56835,54.5354,6298d96e-9b2e-4a00-8036-fa552f9983f5,"POLYGON ((423496.602 524724.298, 423497.204 52..."
3,TLC14,Durham,410381,532242,-1.84050,54.6851,5c8b0b6d-86ec-40a2-9620-b6b8961b575b,"POLYGON ((416954.896 558226.798, 416959.398 55..."
4,TLC21,Northumberland,395323,600699,-2.07521,55.3004,8b993081-f540-458a-8672-652805c2d332,"MULTIPOLYGON (((424965.097 638594.304, 424963...."


In [79]:
# Merging gdf with GDHI df
gdf_GDHI_ITL3_merged = ITL3_gdf.merge(merged_GDHI_ITL3, how='left', 
                                left_on='ITL321CD', right_on='ITL code')

gdf_GDHI_ITL3_merged.head()

,ITL321CD,ITL321NM,BNG_E,BNG_N,LONG,LAT,GlobalID,geometry,ITL code,GDHI_2021,GDHI_perhead_2021
0,TLC11,Hartlepool and Stockton-on-Tees,444952,522016,-1.30587,54.5914,94a02515-1576-4743-8e3d-a843ae72fc4a,"POLYGON ((447213.9 537036.104, 447228.798 5370...",TLC11,5101,17615
1,TLC12,South Teesside,461718,519597,-1.04695,54.5678,f9353bdb-f621-4560-9f58-58c0e888a0c9,"POLYGON ((455834.103 528110.599, 455845.003 52...",TLC12,4672,16664
2,TLC13,Darlington,428029,515649,-1.56835,54.5354,6298d96e-9b2e-4a00-8036-fa552f9983f5,"POLYGON ((423496.602 524724.298, 423497.204 52...",TLC13,1989,18376
3,TLC14,Durham,410381,532242,-1.84050,54.6851,5c8b0b6d-86ec-40a2-9620-b6b8961b575b,"POLYGON ((416954.896 558226.798, 416959.398 55...",TLC14,8982,17228
4,TLC21,Northumberland,395323,600699,-2.07521,55.3004,8b993081-f540-458a-8672-652805c2d332,"MULTIPOLYGON (((424965.097 638594.304, 424963....",TLC21,6553,20380


In [80]:
# Call folium_map function for GDHI_ITL2
GDHI_ITL3_map = folium_map(df= gdf_GDHI_ITL3_merged, 
                          colormap_caption='GDHI_values in millions',
                          key_value={'ITL321NM':'GDHI_2021'},
                          filename='UK_GDHI_2021_ITL3.html')

GDHI_ITL3_map

Map saved in the directory.


In [81]:
# Call folium_map function for GDHI_perhead_ITL2
GDHI_perhead_ITL3_map = folium_map(df= gdf_GDHI_ITL3_merged, 
                          colormap_caption='GDHI_values in millions',
                          key_value={'ITL321NM':'GDHI_perhead_2021'},
                          filename='UK_GDHI_perhead_2021_ITL3.html')

GDHI_perhead_ITL3_map

Map saved in the directory.


In [82]:
# Using psutil
process = psutil.Process()
memory_usage_bytes = process.memory_info().rss

# Convert bytes to MB
memory_usage_mb = memory_usage_bytes / 1024
print(f"Memory usage of the current Python process: \n{memory_usage_mb:.2f} KB")

Memory usage of the current Python process: 
3133764.00 KB


In [83]:
# Using tracemalloc to get the current and peak memory usage in bytes
current, peak = tracemalloc.get_traced_memory()

current_kb = current / 1024
peak_kb = peak / 1024

print(f"Current memory usage: {current_kb:.2f} KB"
      f"\nPeak memory usage: {peak_kb:.2f} KB")

Current memory usage: 639145.07 KB
Peak memory usage: 2100540.66 KB


In [84]:
# Stop tracking memory allocations
tracemalloc.stop()

In [85]:
# ITL code Matching Patterns
ENGLAND = r"^TLB$"
SCOTLAND = r"^TLM$"
WALES = r"^TLL$"
NI = r"^TLN$"

ENGLAND_REGIONS = r"^TL[A-Za-z]$"
NORTH_EAST = r"^TLC(\d{0,2}|[A-Za-z])$"
NORTH_WEST = r"^TLD(\d{0,2}|[A-Za-z])$"
YORKSHIRE_AND_THE_HUMBER = r"^TLE(\d{0,2}|[A-Za-z])$"
EAST_MIDLANDS = r"^TLF(\d{0,2}|[A-Za-z])$"
WEST_MIDLANDS = r"^TLG(\d{0,2}|[A-Za-z])$"
EAST = r"^TLH(\d{0,2}|[A-Za-z])$"
LONDON = r"^TLI(\d{0,2}|[A-Za-z])$"
SOUTH_EAST = r"^TLJ(\d{0,2}|[A-Za-z])$"
SOUTH_WEST = r"^TLK(\d{0,2}|[A-Za-z])$"

TOTAL_WALES = r"^TLL(\d{0,2}|[A-Za-z])$"
WEST_WALES_AND_THE_VALLEYS = r"^TLL1(\d{0,2}|[A-Za-z])$"
EAST_WALES = r"^TLL2(\d{0,2}|[A-Za-z])$"

TOTAL_SCOTLAND = r"^TLM(\d{0,2}|[A-Za-z])$"
NE_SCOTLAND = r"^TLM5(\d{0,2}|[A-Za-z])$"
HIGHLANDS_AND_ISLANS = r"^TLM6(\d{0,2}|[A-Za-z])$"
EASTERN_SCOTLAND = r"^TLM7(\d{0,2}|[A-Za-z])$"
WEST_CENTRAL_SCOTLAND = r"^TLM8(\d{0,2}|[A-Za-z])$"
SOUTHERN_SCOTLAND = r"^TLM9(\d{0,2}|[A-Za-z])$"

TOTAL_NI = r"^TLN(\d{0,2}|[A-Za-z]|\d{0,2}[A-Za-z])$"
